# Reproducing MRIQC report

In [1]:
from bids import BIDSLayout
from nireports.assembler.report import Report

In [2]:
layout = BIDSLayout("bids_testset")

In [3]:
!ls bids_testset/sub-pp001/

ses-01


In [4]:
in_file = layout.get_file("sub-pp001/ses-01/anat/sub-pp001_ses-01_T1w.nii.gz")

In [5]:
entities = in_file.get_entities()
entities.pop("extension", None)
report_type = entities.pop("datatype", None)

In [6]:
print(entities)
print(report_type)

{'session': '01', 'subject': 'pp001', 'suffix': 'T1w'}
anat


In [7]:
bootstrap_file=f"bootstrap_data/bootstrap-{report_type}.yml"

In [8]:
prov = {}
prov["Input filename"] = f"Test filename"
prov["Version Freesurfer"] = f"Version X.Y"
prov["Execution environment"] = f"Test environment"

In [9]:
bids_meta = layout.get_file(in_file).get_metadata()

In [10]:
metadata={
    "dataset": "test name",
    "about-metadata": {
        "Provenance Information": prov,
        "Dataset Information": bids_meta,
    }
}

In [11]:
output_dir = "bids_testset/derivatives/testreport"
uuid = "test_uuid"
reportlets_dir = "bids_testset/derivatives/mriqc/sub-pp001/figures/"

In [12]:
robj = Report(output_dir, uuid, reportlets_dir=reportlets_dir,
              bootstrap_file=bootstrap_file, metadata=metadata,
              plugin_meta={}, **entities)

In [14]:
robj.generate_report()

0

# Apply nireports to Freesurfer output data

## Steps

1. Create figures as either .svg or .html in a `reportlets_dir`

Note: each figure is a "reportlet"

2. Create a bootstrap_file which will gather the appropriate reportlets in sections

3. Create dictionaries for each set of textual information (e.g. brain region volumes)

Note: each set title `set_title` and asscociated dictionary `dict` are to be the values of `metadata["about-metadata"]` so that `metadata["about-metadata"]["set_title'] = dict`

4. Generate the report by generating an instance of `Report` and calling the `generate_report` method